# Deployment

In [ ]:
import os
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

StopWords = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import joblib


# Load the pipeline
pipeLine = joblib.load('model_pipeline.joblib')



In [ ]:
class TextPreprocessor:
    def __init__(self):
        self.StopWords = set(stopwords.words('arabic'))
        self.ArabicDiacritics = re.compile(r"""
                                 ّ    | # Tashdid
                                 َ    | # Fatha
                                 ً    | # Tanwin Fath
                                 ُ    | # Damma
                                 ٌ    | # Tanwin Damm
                                 ِ    | # Kasra
                                 ٍ    | # Tanwin Kasr
                                 ْ    | # Sukun
                                 ـ     # Tatwil/Kashida
                             """, re.VERBOSE)
        self.RegrexPattern = re.compile(
            r"[\U0001F600-\U0001F64F" +  # emoticons {😀 , 😆}
            r"\U0001F300-\U0001F5FF" +  # symbols & pictographs {🌍 , 🌞}
            r"\U0001F680-\U0001F6FF" +  # transport & map symbols {🚌 , 🚕 }
            r"\U0001F1E0-\U0001F1FF]",   # flags (iOS) { 🇺🇸 , 🇨🇦 }
            re.UNICODE
        )


    def preprocess_text(self, text):
        # Remove special characters {& $ @} and punctuation {. , ? !}
        text = re.sub(r'[^\w\s]', '', text)

        # Remove emoji characters
        text = re.sub(self.RegrexPattern, '', text)

        # Remove Arabic diacritics
        text = re.sub(self.ArabicDiacritics, '', text)
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word not in self.StopWords]
        return ' '.join(tokens)

In [ ]:
# Initialize the TextPreprocessor class
preprocessor = TextPreprocessor()

In [ ]:
def classifyArticle(ArticleText, PipeLine):
    # Preprocess the texts
    preprocessedText = preprocessor.preprocess_text(ArticleText)

    PredictedCategory = PipeLine.predict([preprocessedText])[0]

    return PredictedCategory

In [ ]:
category_mapping = {
    0: 'Culture',
    1: 'Finance',
    2: 'Medical',
    3: 'Politics',
    4: 'Religion',
    5: 'Sports',
    6: 'Tech'

}

# Test

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/arabic_categorization_data.csv')

# Display the DataFrame
print(df)
df['type'].unique()


# Sample 3 rows
sampled_df = df.sample(n=5, random_state=1)

# Sample 3 rows
sampled_df2 = df.sample(n=5, random_state=10)

       Unnamed: 0                                               text  \
0               0  \nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...   
1               1  \nتحصل كتاب "المصحف وقراءاته" الذي ألفه باحثون...   
2               2  \nاستنكرت إدارة المسرح الوطني التونسي الحملة ا...   
3               3  \nاحتضن جناح تونس في القرية الدولية للأفلام بم...   
4               4  \nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...   
...           ...                                                ...   
10361        4814  \nأكد المكلف بالإعلام والاتصال بالإدارة العامة...   
10362        4815  \nتمكّنت فرقة الأبحاث والتفتيش للحرس الوطني بط...   
10363        4816  \nقرر الأهالي بمناطق هيشر وعين القارصي والغولا...   
10364        4817  \nتمكنت وحدات الحرس الوطني بمحطة الإستخلاص ببر...   
10365        4818  \nأصيب مساء اليوم الأربعاء، عسكري بطلق ناري عل...   

            type  
0        culture  
1        culture  
2        culture  
3        culture  
4        culture  
...          ...  
10

In [ ]:
df['type'].unique()

array(['culture', 'diverse', 'economy', 'internationalNews', 'politic',
       'sport', 'technology', 'society', 'localnews'], dtype=object)

In [ ]:

# Apply classification and map categories
for index, row in sampled_df.iterrows():
    article = row['text']
    PredictedCategory = classifyArticle(article, pipeLine)
    category_name = category_mapping.get(PredictedCategory, "Unknown Category")
    print(f"Article: {article}")
    print("Predicted category:", category_name)
    print()

Article: 
نفّذ اليوم عدد من أهالي القصرين وقفة احتجاجية في ساحة الشهداء وسط  مدينة القصرين وذلك تحت شعار "نحب نخدم.. نحب نداوي في القصرين".
ويطالب المحتجون بحقهم في التنمية والتشغيل و تسوية وضعيات عمال الحضائر و تحسين أوضاعهم الاجتماعية، وفق ما أكده مراسل "الجوهرة أف أم".

Predicted category: Politics

Article: 
يواجه النادي الإفريقي في الدور السادس عشر من مسابقة كأس الكنفيديرالية الإفريقية لكرة القدم فريق القوات المسلحة السيراليونية.
و كان نادي القوات المسلحة قد فازت في الذهاب على ويكي توريستس النيجيري بهدفين لصفر و إنهزم في الإياب بهدف مقابل صفر . و تجرى مقابلة الذهاب في تونس في الفترة بين 10 و 12 مارس في حين تدور مقابلة إياب بعد أسبوع في سيراليون . يشار أن النادي الصفاقسي سيواجه خلال نفس الدور فريق يانغ سبورت الكاميروني . 

Predicted category: Sports

Article: 
وصف الإعلامي زياد الهاني تصريحات القيادي بحزب نداء تونس، لزهر الكرمي والتي اتهمه فيها بأنه يعمل على تنفيذ أجندا خاصة بنبيل القروي وشق من نداء تونس، بالكلام المخجل وغير المسؤول.
وأضاف الهاني في مداخلة له اليوم في برنامج "بوليت

In [ ]:


#PredictedCategory = classifyArticle(article, pipeLine)

#PredictedCategory = category_mapping.get(PredictedCategory, "Unknown Category")

#print("Predicted category:", PredictedCategory)


Predicted category: Sports
